In [1]:
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/My_project


In [ ]:
from Utils.embedding import *

In [ ]:
#On récupère les noms, on créé des objets pokemon avec, on récupère les types et leurs one-hot encoding et on a notre dataset.
import json
with open("data.json", "r") as f:
    data = json.load(f)

# 1) Noms
pokemon_names = list(data.keys())
#print(pokemon_names)

In [ ]:
from poke_env.environment.pokemon import Pokemon
# 2) transormer en objets
pokemons_list = [Pokemon(gen=8, species=i) for i in pokemon_names]

In [ ]:
pokemon_types_list = [obtain_one_hot_vector_type(p) for p in pokemons_list]

In [ ]:
class TypeAutoencoder(nn.Module):
    def __init__(self, encoded_size=4):  # taille compressée ici
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(18, 12),
            nn.ReLU(),
            nn.Linear(12, encoded_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoded_size, 12),
            nn.ReLU(),
            nn.Linear(12, 18),
            nn.Sigmoid()  # si tu normalises les entrées
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
#Entrainement autoencodeur
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 1. Prépare tes données
# Supposons que `vectors` est ta liste de vecteurs one-hot (chacun de taille 18)
vectors_tensor = torch.tensor(pokemon_types_list, dtype=torch.float32)
dataset = TensorDataset(vectors_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 2. Initialise ton modèle, ta loss, ton optimizer
encodeur_type = TypeAutoencoder(encoded_size=4) #Ou 3 ?
criterion = nn.MSELoss()
optimizer = optim.Adam(encodeur_type.parameters(), lr=1e-3)

# 3. Entraînement
epochs = 50
for epoch in range(epochs):
    total_loss = 0.0
    for batch in dataloader:
        inputs = batch[0]  # batch est un tuple (input,)
        optimizer.zero_grad()
        outputs = encodeur_type(inputs)
        loss = criterion(outputs, inputs)  # reconstruction loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")

In [ ]:
torch.save(encodeur_type.state_dict(), "type_autoencoder.pth")